In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.25, upper_percentile=0.75):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [3]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [4]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [5]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [6]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'].str.contains('segment1')]
    segment2 = results.loc[results['segment'].str.contains('segment2')]
    segment3 = results.loc[results['segment'].str.contains('segment3')]
    segment4 = results.loc[results['segment'].str.contains('segment4')]
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    display(segment4['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [7]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'].str.contains('segment1')]
    segment2 = results.loc[results['segment'].str.contains('segment2')]
    segment3 = results.loc[results['segment'].str.contains('segment3')]
    segment4 = results.loc[results['segment'].str.contains('segment4')]
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Total Savings

In [8]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'].str.contains('segment1')]
    segment2 = results.loc[results['segment'].str.contains('segment2')]
    segment3 = results.loc[results['segment'].str.contains('segment3')]
    segment4 = results.loc[results['segment'].str.contains('segment4')]
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [9]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'].str.contains('segment1')]
    segment2 = results.loc[results['segment'].str.contains('segment2')]
    segment3 = results.loc[results['segment'].str.contains('segment3')]
    segment4 = results.loc[results['segment'].str.contains('segment4')]
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [10]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'].str.contains(segment)]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [11]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'].str.contains('segment1')]
    segment2 = results.loc[results['segment'].str.contains('segment2')]
    segment3 = results.loc[results['segment'].str.contains('segment3')]
    segment4 = results.loc[results['segment'].str.contains('segment4')]
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    print('segment4')
    display(segment4_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']])

In [12]:
community = 'jackson_county'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [13]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Heat Pump Water Heater                                              3269.1
Basic Enclosure                                                     4855.9
Enhanced Enclosure                                                  6941.2
Mininum Efficiency Heat Pump with Existing Heat Backup             16880.9
Mininum Efficiency Heat Pump with Electric Heat Backup             17362.6
Mininum Efficiency Whole Home Electrification                      17468.6
High Efficiency Heat Pump with Electric Heat Backup                28856.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        33594.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     34484.3
High Efficie

## 1.2 Total Annual Savings

In [14]:
figure_total_savings_1(community_results)

In [15]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [16]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [17]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Mininum Efficiency Whole Home Electrification                      17646.462117
Electric Clothes Dryer                                              3591.614938
Electric Cooking                                                    9265.180107
Induction Cooking                                                  27538.893286
Heat Pump Clothes Dryer                                            23302.660882
Mininum Efficiency Heat Pump with Electric Heat Backup             27468.480932
Heat Pump Water Heater                                             27540.061448
Mininum Efficiency Heat Pump with Existing Heat Backup             26996.819723
Basic Enclosure                                                    27181.016959
Enhanced Enclosure                                                 27284.784354
High Efficiency Heat Pump with Electric Heat Backup                27468.480932
High Efficiency Whole Home Electrification                         27723.000000
Basic Enclosure + HPWH + Hi

In [18]:
metric = "pct_saving_energy.total_%"
print("Community summary:")
get_summary(community_results, metric, lower_percentile=0.25, upper_percentile=0.75)

Community summary:


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1147,27723.000000,60.443740,51.499451,71.840000
Basic Enclosure + High Efficiency Whole Home Electrification,1147,27723.000000,59.730811,51.206857,70.990000
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1147,27723.000000,59.121530,50.156316,70.809326
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1147,27723.000000,58.408671,49.738099,69.646394
High Efficiency Whole Home Electrification,1147,27723.000000,54.724767,45.854252,66.460000
High Efficiency Heat Pump with Electric Heat Backup,1144,27468.480932,44.487205,35.440724,57.688596
Mininum Efficiency Whole Home Electrification,890,17646.462117,33.507318,17.563753,52.159270
Mininum Efficiency Heat Pump with Electric Heat Backup,1144,27468.480932,25.035290,10.716305,40.130452
Enhanced Enclosure,1137,27284.784354,20.587384,11.210859,29.619776


# 2. Results for Segments

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, renters, 0-80% AMI 

Segment 2. Occupied, All housing types, renters, 0-80% AMI  

Segment 3. Occupied, Single family detached, pre 1940, renters, all income

Segment 4. Occupied, All housing types, renters, all income  


In [19]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

In [20]:
income = 'specified_by_community'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Single-Family Attached', 'Single-Family Detached', 'Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units', 'Mobile Home']
seg2_vintage = ['<1940', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
seg2_tenure = ['Renter']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['<1940']
seg3_tenure = ['Renter']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Single-Family Attached', 'Single-Family Detached', 'Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units', 'Mobile Home']
seg4_vintage = ['<1940', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
seg4_tenure = ['Renter']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

community_results['segment'] = ""
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg1_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] += 'segment1,'

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg2_tenure) &
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] += 'segment2,'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg3_tenure) &
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] += 'segment3,'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg4_tenure) &
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] += 'segment4'

In [21]:
metric = 'pct_saving_energy.total_%'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    display(get_summary(df_seg, metric, lower_percentile=0.25, upper_percentile=0.75))

Summary for segment: : 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,900,20396.059071,62.905188,55.996054,73.397118
Basic Enclosure + High Efficiency Whole Home Electrification,900,20396.059071,62.128069,54.442909,72.597433
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,900,20396.059071,61.778012,54.689768,72.366122
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,900,20396.059071,61.001017,53.162358,71.582657
High Efficiency Whole Home Electrification,900,20396.059071,57.016020,50.323015,67.608564
High Efficiency Heat Pump with Electric Heat Backup,900,20396.059071,47.500794,40.071411,59.943748
Mininum Efficiency Whole Home Electrification,724,13568.496382,35.189062,19.243330,53.227834
Mininum Efficiency Heat Pump with Electric Heat Backup,900,20396.059071,27.187391,13.320097,42.373276
Enhanced Enclosure,893,20078.873471,22.263074,12.895773,31.606355


Summary for segment: segment1,segment2,segment3,segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,19,463.351509,52.733706,49.608539,75.782593
Basic Enclosure + High Efficiency Whole Home Electrification,19,463.351509,50.345967,49.179037,74.799928
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,19,463.351509,49.708426,48.477582,74.409214
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,19,463.351509,47.318273,48.048080,73.603848
High Efficiency Whole Home Electrification,19,463.351509,37.474530,42.469967,70.154097
Enhanced Enclosure,19,463.351509,23.688295,14.195216,33.190267
High Efficiency Heat Pump with Electric Heat Backup,19,463.351509,22.991845,29.985281,59.874161
Mininum Efficiency Heat Pump with Existing Heat Backup,18,385.037949,21.014582,8.230494,32.916801
Basic Enclosure,19,463.351509,20.128847,13.571993,26.444974


Summary for segment: segment2,segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,176,5720.212562,54.042515,43.585637,64.482950
Basic Enclosure + High Efficiency Whole Home Electrification,176,5720.212562,53.599619,43.585637,64.460366
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,176,5720.212562,52.348907,42.832916,63.267386
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,176,5720.212562,51.906075,42.832916,62.679529
High Efficiency Whole Home Electrification,176,5720.212562,49.573375,38.941592,61.057075
High Efficiency Heat Pump with Electric Heat Backup,175,5661.477392,35.941115,25.550282,47.767322
Mininum Efficiency Whole Home Electrification,106,2963.465092,32.746655,7.659317,49.319999
Mininum Efficiency Heat Pump with Electric Heat Backup,175,5661.477392,20.191775,7.081359,32.758031
Enhanced Enclosure,174,5677.496075,14.978374,7.439878,21.589896


Summary for segment: segment3,segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,6,158.539136,65.755449,63.770000,64.610313
Basic Enclosure + High Efficiency Whole Home Electrification,6,158.539136,65.666875,63.770000,64.610313
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,6,158.539136,62.074759,59.110000,64.093925
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,6,158.539136,61.986185,59.110000,64.093925
High Efficiency Whole Home Electrification,6,158.539136,60.302746,57.743634,60.581855
High Efficiency Heat Pump with Electric Heat Backup,6,158.539136,46.197527,43.240000,48.636415
Mininum Efficiency Heat Pump with Electric Heat Backup,6,158.539136,33.370029,31.472321,32.793265
Mininum Efficiency Heat Pump with Existing Heat Backup,6,158.539136,30.403736,19.560483,33.637090
Enhanced Enclosure,6,158.539136,24.029392,21.140000,28.672057


Summary for segment: segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,46,984.837723,49.419459,39.032873,68.428391
Basic Enclosure + High Efficiency Whole Home Electrification,46,984.837723,49.154933,39.032873,68.353021
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,46,984.837723,47.396216,37.542267,66.488598
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,46,984.837723,47.131875,37.542267,66.477627
High Efficiency Whole Home Electrification,46,984.837723,44.411523,37.088066,63.557607
High Efficiency Heat Pump with Electric Heat Backup,44,789.053825,40.186995,28.886067,50.344970
Mininum Efficiency Whole Home Electrification,37,561.134363,29.562707,2.830571,49.060041
Mininum Efficiency Heat Pump with Electric Heat Backup,44,789.053825,22.286517,8.310510,33.886731
Enhanced Enclosure,45,906.524164,16.413931,7.541550,19.510995


## 2.1 Upgrade Cost Per Dwelling Unit 

In [22]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Heat Pump Water Heater                                              3224.1
Basic Enclosure                                                     7173.8
Enhanced Enclosure                                                 10300.2
Mininum Efficiency Heat Pump with Existing Heat Backup             21191.5
Mininum Efficiency Heat Pump with Electric Heat Backup             28283.9
Mininum Efficiency Whole Home Electrification                      29977.1
High Efficiency Heat Pump with Electric Heat Backup                34412.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        37835.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     38913.9
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Basic Enclosure                                                     2692.2
Heat Pump Water Heater                                              3035.0
Enhanced Enclosure                                                  4108.7
Mininum Efficiency Heat Pump with Existing Heat Backup             12215.2
Mininum Efficiency Heat Pump with Electric Heat Backup             12926.3
Mininum Efficiency Whole Home Electrification                      13687.5
High Efficiency Heat Pump with Electric Heat Backup                25345.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28866.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     29630.4
High Efficie

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Heat Pump Water Heater                                              3157.9
Basic Enclosure                                                     6598.0
Enhanced Enclosure                                                  8984.9
Mininum Efficiency Heat Pump with Existing Heat Backup             22367.8
Mininum Efficiency Whole Home Electrification                      23543.5
Mininum Efficiency Heat Pump with Electric Heat Backup             27504.0
High Efficiency Heat Pump with Electric Heat Backup                32211.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35146.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35996.6
Basic Enclos

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Basic Enclosure                                                     2799.2
Heat Pump Water Heater                                              3033.3
Enhanced Enclosure                                                  4116.6
Mininum Efficiency Heat Pump with Existing Heat Backup             12396.8
Mininum Efficiency Whole Home Electrification                      13043.0
Mininum Efficiency Heat Pump with Electric Heat Backup             13123.4
High Efficiency Heat Pump with Electric Heat Backup                25497.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28382.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     29081.2
High Efficie

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [23]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.05497
Enhanced Enclosure                                                 0.04447
Heat Pump Water Heater                                             0.02664
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02175
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02004
Basic Enclosure + High Efficiency Whole Home Electrification       0.01959
High Efficiency Whole Home Electrification                         0.01333
Heat Pump Clothes Dryer                                            0.01187
High Efficiency Heat Pump with Electric Heat Backup                0.01163
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00745
Induction Cooking                                                 -0.00487
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.01190
Mininum Effi

segment2


upgrade_name
Basic Enclosure                                                    0.08330
Enhanced Enclosure                                                 0.06426
Heat Pump Water Heater                                             0.03151
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02454
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02435
Basic Enclosure + High Efficiency Whole Home Electrification       0.02228
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02216
High Efficiency Whole Home Electrification                         0.01927
High Efficiency Heat Pump with Electric Heat Backup                0.01804
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01486
Heat Pump Clothes Dryer                                            0.00950
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00590
Induction Cooking                                                 -0.00108
Mininum Effi

segment3


upgrade_name
Basic Enclosure                                                    0.06688
Enhanced Enclosure                                                 0.05457
Heat Pump Water Heater                                             0.03917
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02799
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02776
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02513
Basic Enclosure + High Efficiency Whole Home Electrification       0.02487
High Efficiency Whole Home Electrification                         0.01853
High Efficiency Heat Pump with Electric Heat Backup                0.01728
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01335
Heat Pump Clothes Dryer                                            0.00693
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00387
Induction Cooking                                                 -0.00452
Mininum Effi

segment4


upgrade_name
Basic Enclosure                                                    0.08714
Enhanced Enclosure                                                 0.06845
Heat Pump Water Heater                                             0.03328
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02617
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02597
Basic Enclosure + High Efficiency Whole Home Electrification       0.02363
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02351
High Efficiency Whole Home Electrification                         0.02045
High Efficiency Heat Pump with Electric Heat Backup                0.01931
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01564
Heat Pump Clothes Dryer                                            0.00905
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00709
Induction Cooking                                                 -0.00110
Mininum Effi

 ### Annual Energy Savings Per Dollar Invested

In [24]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.00387
Enhanced Enclosure                                                 0.00323
Heat Pump Water Heater                                             0.00264
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00207
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00204
Basic Enclosure + High Efficiency Whole Home Electrification       0.00190
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00188
High Efficiency Whole Home Electrification                         0.00173
Mininum Efficiency Whole Home Electrification                      0.00171
High Efficiency Heat Pump with Electric Heat Backup                0.00166
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00147
Electric Cooking                                                   0.00146
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00127
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.00436
Enhanced Enclosure                                                 0.00335
Mininum Efficiency Whole Home Electrification                      0.00247
Heat Pump Water Heater                                             0.00191
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00142
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00141
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00139
Electric Cooking                                                   0.00138
Basic Enclosure + High Efficiency Whole Home Electrification       0.00129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00127
High Efficiency Whole Home Electrification                         0.00122
High Efficiency Heat Pump with Electric Heat Backup                0.00116
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00094
Electric Clo

segment3


upgrade_name
Basic Enclosure                                                    0.00406
Enhanced Enclosure                                                 0.00345
Heat Pump Water Heater                                             0.00267
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00210
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00208
Basic Enclosure + High Efficiency Whole Home Electrification       0.00192
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00190
Mininum Efficiency Whole Home Electrification                      0.00177
High Efficiency Whole Home Electrification                         0.00173
High Efficiency Heat Pump with Electric Heat Backup                0.00168
Electric Cooking                                                   0.00155
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00135
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00127
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.00444
Enhanced Enclosure                                                 0.00350
Mininum Efficiency Whole Home Electrification                      0.00254
Heat Pump Water Heater                                             0.00195
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00148
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00147
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00145
Electric Cooking                                                   0.00138
Basic Enclosure + High Efficiency Whole Home Electrification       0.00134
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00133
High Efficiency Whole Home Electrification                         0.00127
High Efficiency Heat Pump with Electric Heat Backup                0.00121
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00096
Electric Clo

 ### Annual Emission Savings Per Dollar Invested

In [25]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.27313
Enhanced Enclosure                                                 0.22847
Heat Pump Water Heater                                             0.17944
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14114
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13989
Basic Enclosure + High Efficiency Whole Home Electrification       0.12960
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12877
High Efficiency Whole Home Electrification                         0.11481
High Efficiency Heat Pump with Electric Heat Backup                0.10931
Electric Cooking                                                   0.09719
Mininum Efficiency Heat Pump with Existing Heat Backup             0.09671
Mininum Efficiency Whole Home Electrification                      0.09530
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06964
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.32305
Enhanced Enclosure                                                 0.24980
Mininum Efficiency Whole Home Electrification                      0.14448
Heat Pump Water Heater                                             0.13310
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.10209
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.10057
Basic Enclosure + High Efficiency Whole Home Electrification       0.09314
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.09199
Mininum Efficiency Heat Pump with Electric Heat Backup             0.09010
Electric Cooking                                                   0.08857
High Efficiency Whole Home Electrification                         0.08660
High Efficiency Heat Pump with Electric Heat Backup                0.08255
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06846
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.29102
Enhanced Enclosure                                                 0.24744
Heat Pump Water Heater                                             0.18518
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14604
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.14482
Basic Enclosure + High Efficiency Whole Home Electrification       0.13359
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13276
High Efficiency Whole Home Electrification                         0.11783
High Efficiency Heat Pump with Electric Heat Backup                0.11362
Mininum Efficiency Whole Home Electrification                      0.09897
Electric Cooking                                                   0.09888
Mininum Efficiency Heat Pump with Existing Heat Backup             0.09193
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07460
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.32971
Enhanced Enclosure                                                 0.26119
Mininum Efficiency Whole Home Electrification                      0.14796
Heat Pump Water Heater                                             0.13592
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.10638
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.10491
Basic Enclosure + High Efficiency Whole Home Electrification       0.09680
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.09570
Mininum Efficiency Heat Pump with Electric Heat Backup             0.09383
High Efficiency Whole Home Electrification                         0.08990
Electric Cooking                                                   0.08704
High Efficiency Heat Pump with Electric Heat Backup                0.08644
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06969
Heat Pump Cl

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [26]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    403872.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     392245.7
Basic Enclosure + High Efficiency Whole Home Electrification       384922.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        373295.3
High Efficiency Whole Home Electrification                         260790.1
Enhanced Enclosure                                                 212214.8
High Efficiency Heat Pump with Electric Heat Backup                185441.2
Basic Enclosure                                                    182714.6
Mininum Efficiency Heat Pump with Existing Heat Backup              60753.3
Heat Pump Water Heater                                              39801.4
Heat Pump Clothes Dryer                                             14298.1
Electric Clothes Dryer                                              -3608.8
Induction Cooking                                                   -4462.8

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4564308.3
Basic Enclosure + High Efficiency Whole Home Electrification       4483602.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4461316.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4380610.5
High Efficiency Whole Home Electrification                         3790040.1
High Efficiency Heat Pump with Electric Heat Backup                2800952.1
Enhanced Enclosure                                                 1621419.9
Basic Enclosure                                                    1366641.7
Mininum Efficiency Heat Pump with Existing Heat Backup             1093915.8
Heat Pump Water Heater                                              589980.4
Mininum Efficiency Heat Pump with Electric Heat Backup              467187.2
Heat Pump Clothes Dryer                                             100917.4
Induction Cooking                                              

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    634048.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     626476.9
Basic Enclosure + High Efficiency Whole Home Electrification       614377.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        606805.9
High Efficiency Whole Home Electrification                         460246.5
High Efficiency Heat Pump with Electric Heat Backup                346062.6
Enhanced Enclosure                                                 304939.5
Basic Enclosure                                                    274409.7
Mininum Efficiency Heat Pump with Existing Heat Backup             162283.0
Heat Pump Water Heater                                              76915.8
Heat Pump Clothes Dryer                                             11207.7
Induction Cooking                                                   -5564.1
Electric Cooking                                                    -9250.0

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5653229.3
Basic Enclosure + High Efficiency Whole Home Electrification       5561556.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5533317.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5441643.9
High Efficiency Whole Home Electrification                         4700927.2
High Efficiency Heat Pump with Electric Heat Backup                3481325.0
Enhanced Enclosure                                                 2030630.8
Basic Enclosure                                                    1746230.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1389797.9
Heat Pump Water Heater                                              736224.1
Mininum Efficiency Heat Pump with Electric Heat Backup              658006.6
Heat Pump Clothes Dryer                                             117737.5
Induction Cooking                                              

### Annual Total Energy Savings

In [27]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    37882.4
Basic Enclosure + High Efficiency Whole Home Electrification       37334.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     36827.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        36279.6
High Efficiency Whole Home Electrification                         33745.2
High Efficiency Heat Pump with Electric Heat Backup                26486.3
Mininum Efficiency Whole Home Electrification                      20266.5
Mininum Efficiency Heat Pump with Electric Heat Backup             16617.9
Enhanced Enclosure                                                 15436.7
Basic Enclosure                                                    12877.8
Mininum Efficiency Heat Pump with Existing Heat Backup             12017.2
Heat Pump Water Heater                                              3940.4
Heat Pump Clothes Dryer                                              565.3
Induction Co

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    262441.1
Basic Enclosure + High Efficiency Whole Home Electrification       260107.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     254855.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        252521.6
High Efficiency Whole Home Electrification                         240053.7
High Efficiency Heat Pump with Electric Heat Backup                179877.8
Mininum Efficiency Whole Home Electrification                      113583.6
Mininum Efficiency Heat Pump with Electric Heat Backup             112411.0
Enhanced Enclosure                                                  84586.8
Basic Enclosure                                                     71465.3
Mininum Efficiency Heat Pump with Existing Heat Backup              69144.9
Heat Pump Water Heater                                              35854.1
Heat Pump Clothes Dryer                                              3979.3

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    48042.2
Basic Enclosure + High Efficiency Whole Home Electrification       47473.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     46487.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        45918.5
High Efficiency Whole Home Electrification                         43016.8
High Efficiency Heat Pump with Electric Heat Backup                33721.0
Mininum Efficiency Whole Home Electrification                      23102.2
Mininum Efficiency Heat Pump with Electric Heat Backup             21804.6
Enhanced Enclosure                                                 19297.8
Basic Enclosure                                                    16652.4
Mininum Efficiency Heat Pump with Existing Heat Backup             16422.0
Heat Pump Water Heater                                              5244.3
Heat Pump Clothes Dryer                                              797.6
Induction Co

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    318642.8
Basic Enclosure + High Efficiency Whole Home Electrification       315992.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     308978.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        306327.7
High Efficiency Whole Home Electrification                         291107.7
High Efficiency Heat Pump with Electric Heat Backup                218790.1
Mininum Efficiency Heat Pump with Electric Heat Backup             137169.8
Mininum Efficiency Whole Home Electrification                      135261.5
Enhanced Enclosure                                                 103941.7
Basic Enclosure                                                     88895.2
Mininum Efficiency Heat Pump with Existing Heat Backup              84890.2
Heat Pump Water Heater                                              43245.1
Heat Pump Clothes Dryer                                              5034.0

### Annual Total Emission Savings

In [28]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2594885.8
Basic Enclosure + High Efficiency Whole Home Electrification       2547002.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2522257.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2474374.1
High Efficiency Whole Home Electrification                         2245652.8
High Efficiency Heat Pump with Electric Heat Backup                1742921.6
Mininum Efficiency Whole Home Electrification                      1127989.8
Enhanced Enclosure                                                 1090398.0
Mininum Efficiency Heat Pump with Electric Heat Backup              912713.7
Basic Enclosure                                                     907891.0
Mininum Efficiency Heat Pump with Existing Heat Backup              789110.5
Heat Pump Water Heater                                              268062.5
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    18945054.2
Basic Enclosure + High Efficiency Whole Home Electrification       18741842.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     18426588.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        18223376.7
High Efficiency Whole Home Electrification                         17033959.2
High Efficiency Heat Pump with Electric Heat Backup                12815755.1
Mininum Efficiency Heat Pump with Electric Heat Backup              7133462.3
Mininum Efficiency Whole Home Electrification                       6641233.1
Enhanced Enclosure                                                  6302663.6
Basic Enclosure                                                     5299865.3
Mininum Efficiency Heat Pump with Existing Heat Backup              5038323.8
Heat Pump Water Heater                                              2492288.0
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3349936.0
Basic Enclosure + High Efficiency Whole Home Electrification       3300204.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3241854.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3192122.4
High Efficiency Whole Home Electrification                         2927159.2
High Efficiency Heat Pump with Electric Heat Backup                2275955.1
Enhanced Enclosure                                                 1382608.0
Mininum Efficiency Whole Home Electrification                      1289453.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1275941.5
Basic Enclosure                                                    1194143.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1117701.3
Heat Pump Water Heater                                              363660.6
Heat Pump Clothes Dryer                                        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    23011158.3
Basic Enclosure + High Efficiency Whole Home Electrification       22780304.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     22352831.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        22121976.9
High Efficiency Whole Home Electrification                         20660343.9
High Efficiency Heat Pump with Electric Heat Backup                15586783.9
Mininum Efficiency Heat Pump with Electric Heat Backup              8708916.1
Mininum Efficiency Whole Home Electrification                       7869887.5
Enhanced Enclosure                                                  7748107.5
Basic Enclosure                                                     6607266.9
Mininum Efficiency Heat Pump with Existing Heat Backup              6192944.1
Heat Pump Water Heater                                              3006962.7
Heat Pump Clothes Dryer                            

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [29]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    871.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     846.5
Basic Enclosure + High Efficiency Whole Home Electrification       830.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        805.6
High Efficiency Whole Home Electrification                         562.8
Enhanced Enclosure                                                 458.0
High Efficiency Heat Pump with Electric Heat Backup                400.2
Basic Enclosure                                                    394.3
Mininum Efficiency Heat Pump with Existing Heat Backup             157.8
Heat Pump Water Heater                                              85.9
Heat Pump Clothes Dryer                                             30.9
Induction Cooking                                                   -9.6
Electric Cooking                                                   -30.0
Electric Clothes Dryer                

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    738.1
Basic Enclosure + High Efficiency Whole Home Electrification       725.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     721.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        708.4
High Efficiency Whole Home Electrification                         612.9
High Efficiency Heat Pump with Electric Heat Backup                457.3
Enhanced Enclosure                                                 264.0
Basic Enclosure                                                    224.3
Mininum Efficiency Heat Pump with Existing Heat Backup             181.6
Heat Pump Water Heater                                              95.6
Mininum Efficiency Heat Pump with Electric Heat Backup              76.3
Heat Pump Clothes Dryer                                             24.7
Induction Cooking                                                   -2.1
Electric Cooking                      

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1019.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1007.4
Basic Enclosure + High Efficiency Whole Home Electrification        987.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         975.7
High Efficiency Whole Home Electrification                          740.1
High Efficiency Heat Pump with Electric Heat Backup                 556.5
Enhanced Enclosure                                                  490.3
Basic Enclosure                                                     441.3
Mininum Efficiency Heat Pump with Existing Heat Backup              298.5
Heat Pump Water Heater                                              123.7
Heat Pump Clothes Dryer                                              18.0
Induction Cooking                                                    -8.9
Electric Cooking                                                    -25.3
Electric Clothes Dryer   

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    771.6
Basic Enclosure + High Efficiency Whole Home Electrification       759.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     755.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        742.7
High Efficiency Whole Home Electrification                         641.6
High Efficiency Heat Pump with Electric Heat Backup                492.2
Enhanced Enclosure                                                 281.8
Basic Enclosure                                                    243.9
Mininum Efficiency Heat Pump with Existing Heat Backup             193.9
Heat Pump Water Heater                                             100.9
Mininum Efficiency Heat Pump with Electric Heat Backup              93.0
Heat Pump Clothes Dryer                                             23.5
Induction Cooking                                                   -2.2
Electric Cooking                      

### Annual Energy Savings Per Dwelling Unit

In [30]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    81.8
Basic Enclosure + High Efficiency Whole Home Electrification       80.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     79.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        78.3
High Efficiency Whole Home Electrification                         72.8
High Efficiency Heat Pump with Electric Heat Backup                57.2
Mininum Efficiency Whole Home Electrification                      51.3
Mininum Efficiency Heat Pump with Electric Heat Backup             35.9
Enhanced Enclosure                                                 33.3
Mininum Efficiency Heat Pump with Existing Heat Backup             31.2
Basic Enclosure                                                    27.8
Heat Pump Water Heater                                              8.5
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    42.4
Basic Enclosure + High Efficiency Whole Home Electrification       42.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     41.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        40.8
High Efficiency Whole Home Electrification                         38.8
Mininum Efficiency Whole Home Electrification                      33.8
High Efficiency Heat Pump with Electric Heat Backup                29.4
Mininum Efficiency Heat Pump with Electric Heat Backup             18.4
Enhanced Enclosure                                                 13.8
Basic Enclosure                                                    11.7
Mininum Efficiency Heat Pump with Existing Heat Backup             11.5
Heat Pump Water Heater                                              5.8
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    77.3
Basic Enclosure + High Efficiency Whole Home Electrification       76.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     74.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        73.8
High Efficiency Whole Home Electrification                         69.2
High Efficiency Heat Pump with Electric Heat Backup                54.2
Mininum Efficiency Whole Home Electrification                      41.7
Mininum Efficiency Heat Pump with Electric Heat Backup             35.1
Enhanced Enclosure                                                 31.0
Mininum Efficiency Heat Pump with Existing Heat Backup             30.2
Basic Enclosure                                                    26.8
Heat Pump Water Heater                                              8.4
Electric Cooking                                                    1.8
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    43.5
Basic Enclosure + High Efficiency Whole Home Electrification       43.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     42.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        41.8
High Efficiency Whole Home Electrification                         39.7
Mininum Efficiency Whole Home Electrification                      33.2
High Efficiency Heat Pump with Electric Heat Backup                30.9
Mininum Efficiency Heat Pump with Electric Heat Backup             19.4
Enhanced Enclosure                                                 14.4
Basic Enclosure                                                    12.4
Mininum Efficiency Heat Pump with Existing Heat Backup             11.8
Heat Pump Water Heater                                              5.9
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [31]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    52.7
Basic Enclosure + High Efficiency Whole Home Electrification       50.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     49.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        47.3
High Efficiency Whole Home Electrification                         37.5
Enhanced Enclosure                                                 23.7
High Efficiency Heat Pump with Electric Heat Backup                23.0
Mininum Efficiency Heat Pump with Existing Heat Backup             21.0
Basic Enclosure                                                    20.1
Heat Pump Water Heater                                              8.2
Electric Cooking                                                    2.9
Induction Cooking                                                   1.7
Heat Pump Clothes Dryer                                             1.3
Electric Clothes Dryer                             

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53.9
Basic Enclosure + High Efficiency Whole Home Electrification       53.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     52.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.6
High Efficiency Whole Home Electrification                         48.7
High Efficiency Heat Pump with Electric Heat Backup                35.0
Mininum Efficiency Whole Home Electrification                      28.3
Mininum Efficiency Heat Pump with Electric Heat Backup             18.0
Enhanced Enclosure                                                 15.6
Mininum Efficiency Heat Pump with Existing Heat Backup             13.9
Basic Enclosure                                                    13.4
Heat Pump Water Heater                                              8.6
Electric Cooking                                                    2.3
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    56.1
Basic Enclosure + High Efficiency Whole Home Electrification       54.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     52.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.1
High Efficiency Whole Home Electrification                         43.3
High Efficiency Heat Pump with Electric Heat Backup                28.9
Enhanced Enclosure                                                 23.8
Mininum Efficiency Heat Pump with Existing Heat Backup             23.8
Basic Enclosure                                                    21.0
Heat Pump Water Heater                                              8.4
Electric Cooking                                                    2.7
Mininum Efficiency Heat Pump with Electric Heat Backup              2.0
Induction Cooking                                                   1.8
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53.6
Basic Enclosure + High Efficiency Whole Home Electrification       53.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.2
High Efficiency Whole Home Electrification                         48.3
High Efficiency Heat Pump with Electric Heat Backup                35.8
Mininum Efficiency Whole Home Electrification                      27.9
Mininum Efficiency Heat Pump with Electric Heat Backup             18.8
Enhanced Enclosure                                                 15.9
Basic Enclosure                                                    13.8
Mininum Efficiency Heat Pump with Existing Heat Backup             13.8
Heat Pump Water Heater                                              8.6
Electric Cooking                                                    2.3
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [32]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5600.3
Basic Enclosure + High Efficiency Whole Home Electrification       5496.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5443.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5340.2
High Efficiency Whole Home Electrification                         4846.5
High Efficiency Heat Pump with Electric Heat Backup                3761.6
Mininum Efficiency Whole Home Electrification                      2856.9
Enhanced Enclosure                                                 2353.3
Mininum Efficiency Heat Pump with Existing Heat Backup             2049.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1969.8
Basic Enclosure                                                    1959.4
Heat Pump Water Heater                                              578.5
Electric Cooking                                                    111.8
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3063.8
Basic Enclosure + High Efficiency Whole Home Electrification       3030.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2979.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2947.1
High Efficiency Whole Home Electrification                         2754.7
High Efficiency Heat Pump with Electric Heat Backup                2092.4
Mininum Efficiency Whole Home Electrification                      1977.6
Mininum Efficiency Heat Pump with Electric Heat Backup             1164.7
Enhanced Enclosure                                                 1026.4
Basic Enclosure                                                     869.7
Mininum Efficiency Heat Pump with Existing Heat Backup              836.2
Heat Pump Water Heater                                              404.0
Electric Cooking                                                    101.9
Heat Pump Clothes Dryer  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5386.7
Basic Enclosure + High Efficiency Whole Home Electrification       5306.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5212.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5132.9
High Efficiency Whole Home Electrification                         4706.9
High Efficiency Heat Pump with Electric Heat Backup                3659.7
Mininum Efficiency Whole Home Electrification                      2330.2
Enhanced Enclosure                                                 2223.2
Mininum Efficiency Heat Pump with Existing Heat Backup             2056.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2051.7
Basic Enclosure                                                    1920.2
Heat Pump Water Heater                                              584.8
Electric Cooking                                                    113.7
Heat Pump Clothes Dryer  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3140.6
Basic Enclosure + High Efficiency Whole Home Electrification       3109.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3050.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3019.3
High Efficiency Whole Home Electrification                         2819.8
High Efficiency Heat Pump with Electric Heat Backup                2203.9
Mininum Efficiency Whole Home Electrification                      1929.9
Mininum Efficiency Heat Pump with Electric Heat Backup             1231.4
Enhanced Enclosure                                                 1075.2
Basic Enclosure                                                     922.9
Mininum Efficiency Heat Pump with Existing Heat Backup              863.9
Heat Pump Water Heater                                              412.3
Electric Cooking                                                    100.1
Heat Pump Clothes Dryer  

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [33]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [34]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [35]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [36]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [37]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_84261/850283863.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_84261/850283863.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_84261/850283863.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,12.2,7.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,12.2,7.2
Basic Enclosure + High Efficiency Whole Home Electrification,12.2,7.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,12.2,7.4
High Efficiency Whole Home Electrification,12.2,8.6
High Efficiency Heat Pump with Electric Heat Backup,12.2,9.5
Enhanced Enclosure,12.2,9.6
Basic Enclosure,12.2,10.0
Mininum Efficiency Heat Pump with Existing Heat Backup,14.4,13.8


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,12.1,6.9
Basic Enclosure + High Efficiency Whole Home Electrification,12.1,7.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,12.1,7.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,12.1,7.1
High Efficiency Whole Home Electrification,12.1,7.7
High Efficiency Heat Pump with Electric Heat Backup,12.1,8.9
Enhanced Enclosure,12.1,10.5
Basic Enclosure,11.9,10.5
Mininum Efficiency Heat Pump with Existing Heat Backup,12.1,10.9


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.3,5.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.3,5.8
Basic Enclosure + High Efficiency Whole Home Electrification,10.3,5.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.3,6.0
High Efficiency Whole Home Electrification,10.3,7.0
High Efficiency Heat Pump with Electric Heat Backup,10.3,7.8
Enhanced Enclosure,10.3,8.1
Basic Enclosure,10.3,8.4
Mininum Efficiency Heat Pump with Existing Heat Backup,11.6,10.8


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.7,6.1
Basic Enclosure + High Efficiency Whole Home Electrification,10.7,6.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.7,6.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.7,6.3
High Efficiency Whole Home Electrification,10.7,6.8
High Efficiency Heat Pump with Electric Heat Backup,11.0,8.0
Enhanced Enclosure,10.8,9.3
Basic Enclosure,10.7,9.4
Mininum Efficiency Heat Pump with Existing Heat Backup,10.7,9.6


In [38]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [39]:
! jupyter nbconvert --output-dir='./results/jackson_county/' --to=html --no-input saving_results_reports_for_jackson_county.ipynb

[NbConvertApp] Converting notebook saving_results_reports_for_jackson_county.ipynb to html
/Users/lliu2/micromamba/envs/res-est/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'application/javascript' -%}
[NbConvertApp] Writing 397928 bytes to results/jackson_county/saving_results_reports_for_jackson_county.html
